In [1]:
pip install pandas requests


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
from pathlib import Path
import json
from dataclasses import dataclass, asdict

In [3]:
# URL del endpoint de la API pública de prueba (JSONPlaceholder)
USER_API_URL = "https://jsonplaceholder.typicode.com/users"
WEATHER_API_KEY = "6d3845d7ba9032d63278d8eb8be33f07"

In [4]:
def read_orders_csv(file_name: str = "orders.csv"):
    """
    Lee un archivo CSV del directorio actual y lo carga en un DataFrame de Pandas.

    Args:
        file_name (str): El nombre del archivo CSV a leer. Por defecto, 'orders.csv'.

    Returns:
        pd.DataFrame | None: El DataFrame con los datos de las órdenes, 
                             o None si el archivo no se encuentra o hay un error.
    """
    # 1. Definir la ruta del archivo (asume que está en el mismo directorio)
    file_path = Path.cwd() / file_name  # Path.cwd() obtiene el directorio de trabajo actual
    
    print(f"[*] Intentando leer el archivo: {file_path}")
    
    try:
        # 2. Leer el archivo CSV usando Pandas
        df = pd.read_csv(file_path,sep=r"\s*[;,]\s*",engine="python")
        
        df_sorted = df.sort_values(by="Rep").reset_index(drop=True)
        
        # 3. Reportar éxito
        print(f"[ÉXITO] Archivo '{file_name}' cargado correctamente.")
        print(f"[*] Filas leídas: {len(df_sorted)}. Columnas: {len(df_sorted.columns)}.")
        
        return df_sorted
        
    except FileNotFoundError:
        # Manejo de error si el archivo no existe
        print("-" * 50)
        print(f"[ERROR] Archivo NO encontrado: '{file_name}'")
        print(f"[SUGERENCIA] Asegúrate de que el archivo exista en la ruta: {file_path}")
        return None
        
    except pd.errors.EmptyDataError:
        # Manejo de error si el archivo existe pero está vacío
        print(f"[ERROR] Archivo encontrado, pero está vacío: '{file_name}'")
        return None
        
    except Exception as e:
        # Manejo de otros errores (ej. problemas de codificación)
        print(f"[ERROR INESPERADO] Ocurrió un error al leer el CSV: {e}")
        return None

In [5]:
def read_data_from_file():
    orders_df = read_orders_csv()

    if orders_df is None:
        print("\n" + "=" * 50)
        print("No se pudo proceder con el análisis de datos.")
        print("=" * 50)
        return orders_df

    return orders_df

In [6]:
def fetch_data_from_api_weather(lat: str, lon: str):
    """
    Realiza una solicitud GET a la URL especificada y procesa la respuesta.
    """
    WEATHER_API_URL = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}"
    print(f"[*] Haciendo solicitud GET a: {WEATHER_API_URL}")
    
    try:
        # 1. Realizar la solicitud HTTP GET
        # La solicitud es síncrona: el programa espera aquí hasta que la API responde.
        response = requests.get(WEATHER_API_URL)
        
        # 2. Verificar el estado de la respuesta
        # .raise_for_status() lanzará una excepción (HTTPError) para códigos de error 4xx/5xx
        response.raise_for_status() 
        
        # 3. Extraer y retornar los datos JSON
        # .json() automáticamente parsea la respuesta de texto a un diccionario/lista de Python
        data = response.json()
        
        return data

    except requests.exceptions.HTTPError as err_http:
        # Manejo de errores HTTP (404 Not Found, 500 Internal Server Error, etc.)
        print(f"[ERROR HTTP]: La API devolvió un error. Código: {response.status_code}")
        print(f"Detalles: {err_http}")
        return None
        
    except requests.exceptions.ConnectionError:
        # Manejo de errores de conexión (ej. no hay internet, DNS falló)
        print("[ERROR CONEXIÓN]: No se pudo conectar a la URL. Verifique su conexión y la URL.")
        return None
        
    except requests.exceptions.Timeout:
        # Manejo de errores de timeout
        print("[ERROR TIMEOUT]: La solicitud expiró.")
        return None
        
    except requests.exceptions.RequestException as e:
        # Manejo de cualquier otro error de requests
        print(f"[ERROR INESPERADO]: Ocurrió un error en la solicitud: {e}")
        return None

In [7]:
def fetch_data_from_api_user():
    """
    Realiza una solicitud GET a la URL especificada y procesa la respuesta.
    """
    print(f"[*] Haciendo solicitud GET a: {USER_API_URL}")
    
    try:
        # 1. Realizar la solicitud HTTP GET
        # La solicitud es síncrona: el programa espera aquí hasta que la API responde.
        response = requests.get(USER_API_URL)
        
        # 2. Verificar el estado de la respuesta
        # .raise_for_status() lanzará una excepción (HTTPError) para códigos de error 4xx/5xx
        response.raise_for_status() 
        
        # 3. Extraer y retornar los datos JSON
        # .json() automáticamente parsea la respuesta de texto a un diccionario/lista de Python
        data = response.json()
        
        return data

    except requests.exceptions.HTTPError as err_http:
        # Manejo de errores HTTP (404 Not Found, 500 Internal Server Error, etc.)
        print(f"[ERROR HTTP]: La API devolvió un error. Código: {response.status_code}")
        print(f"Detalles: {err_http}")
        return None
        
    except requests.exceptions.ConnectionError:
        # Manejo de errores de conexión (ej. no hay internet, DNS falló)
        print("[ERROR CONEXIÓN]: No se pudo conectar a la URL. Verifique su conexión y la URL.")
        return None
        
    except requests.exceptions.Timeout:
        # Manejo de errores de timeout
        print("[ERROR TIMEOUT]: La solicitud expiró.")
        return None
        
    except requests.exceptions.RequestException as e:
        # Manejo de cualquier otro error de requests
        print(f"[ERROR INESPERADO]: Ocurrió un error en la solicitud: {e}")
        return None

In [8]:
@dataclass
class Weather:
    main: str
    celsius: float

    #lat=44.34,lon=10.99
    def __init__(self, lat, lon):
        weather_api_response = self.get_weather_data_from_geo(lat=lat,lon=lon)
        local_weather = weather_api_response['current']
        self.main = local_weather['weather'][0]['main']
        self.celsius = self.temp_kelvin_to_celsius(local_weather["temp"])

    @staticmethod
    def temp_kelvin_to_celsius(kelvin):
        celsius = kelvin - 273.15
        return celsius
    
    @staticmethod
    def get_weather_data_from_geo(lat, lon):
        weather_dt = fetch_data_from_api_weather(lat, lon)
        if weather_dt is None:
            print("\n[FIN] No se pudieron obtener los datos del clima.")
            return None
        return weather_dt
    
    def __repr__(self):
        return f"Weather(Main='{self.main}', Celcius='{self.celsius}')"

In [9]:
@dataclass
class Sale:
    units: float
    unit_cost: float
    total: float
    
    def __init__(self, units, unit_cost, total):
        self.units = units
        self.unit_cost = unit_cost
        self.total = total
        
    def sale_calculation(self):
        self.total = self.units * self.unit_cost
        
    def __repr__(self):
        return f"Sale(units='{self.units}', unit_cost='{self.unit_cost}', total='{self.total}')"
    

In [10]:
@dataclass
class SaleSummary:
    total_units_sold: int
    total_revenue: float

    def __init__(self, sale_list: list[Sale]):
        unit_list = []
        total_list = []
        for usuario in sale_list.values.tolist():
            unit_list.append(usuario[4])
            total_list.append(usuario[6])
            
        self.total_units_sold = self.total_units(unit_list)
        self.total_revenue = self.total_revenue(total_list)
        
    def total_units(self, unit_list: list[int]) -> int:
        return sum(unit_list)
    
    def total_revenue(self, total_list: list[float]) -> float:
        return sum(total_list)
        

    def __repr__(self):
        return f"SaleSummary(total_units_sold='{self.total_units_sold}', total_revenue='{self.total_revenue}')"

In [11]:
@dataclass
class Geo:
    lat: str = ""
    lon: str = ""

    def __init__(self, dt):
        self.lat = dt['lat']
        self.lon = dt['lng']

    def __repr__(self):
        return f"Geo(Latitud='{self.lat}', Longitud='{self.lon}')"

In [12]:
@dataclass
class Usuario:
    id: str 
    name: str 
    username: str
    email: str 
    geo: Geo 

    def __init__(self, data):
        self.id = data['id']
        self.name = data['name']
        self.username = data['username']
        self.email = data['email']
        self.geo = Geo(data['address']['geo'])

    def __repr__(self):
        return f"Usuario(Id='{self.id}', Name='{self.name}', Username='{self.username}', Email='{self.email}', Geo='{self.geo.__repr__()}')"


In [13]:
@dataclass
class SalesUsuarios:
    userId: int
    name: str
    username: str
    email: str
    weather: Weather
    sale_summary: SaleSummary

    def __init__(self, usuario: Usuario, sale_summary: SaleSummary ):
        self.userId = usuario.id
        self.name = usuario.name
        self.username = usuario.username
        self.email = usuario.email
        self.weather = Weather(lat=usuario.geo.lat, lon=usuario.geo.lon)
        self.sale_summary = sale_summary
        

    def __repr__(self):
        return f"SalesUsuarios(userId='{self.userId}', name='{self.name}', username='{self.username}', email='{self.email}', weather='{self.weather.__repr__()}', sale_summary='{self.sale_summary.__repr__()}')"

In [14]:

def get_user_data():
    users_data = fetch_data_from_api_user()
    
    if users_data is None:
        print("\n[FIN] No se pudieron obtener los datos de usuario.")
        return None
    
    user_list = [ Usuario(usr) for usr in users_data]
    
    return user_list
        

In [15]:
user_list = get_user_data()
user_list

[*] Haciendo solicitud GET a: https://jsonplaceholder.typicode.com/users


[Usuario(Id='1', Name='Leanne Graham', Username='Bret', Email='Sincere@april.biz', Geo='Geo(Latitud='-37.3159', Longitud='81.1496')'),
 Usuario(Id='2', Name='Ervin Howell', Username='Antonette', Email='Shanna@melissa.tv', Geo='Geo(Latitud='-43.9509', Longitud='-34.4618')'),
 Usuario(Id='3', Name='Clementine Bauch', Username='Samantha', Email='Nathan@yesenia.net', Geo='Geo(Latitud='-68.6102', Longitud='-47.0653')'),
 Usuario(Id='4', Name='Patricia Lebsack', Username='Karianne', Email='Julianne.OConner@kory.org', Geo='Geo(Latitud='29.4572', Longitud='-164.2990')'),
 Usuario(Id='5', Name='Chelsey Dietrich', Username='Kamren', Email='Lucio_Hettinger@annie.ca', Geo='Geo(Latitud='-31.8129', Longitud='62.5342')'),
 Usuario(Id='6', Name='Mrs. Dennis Schulist', Username='Leopoldo_Corkery', Email='Karley_Dach@jasper.info', Geo='Geo(Latitud='-71.4197', Longitud='71.7478')'),
 Usuario(Id='7', Name='Kurtis Weissnat', Username='Elwyn.Skiles', Email='Telly.Hoeger@billy.biz', Geo='Geo(Latitud='24.8918

In [16]:

full_list = read_data_from_file()

andrew_list_trx = full_list[full_list['Rep'] == 'Andrews']
gil_list_trx = full_list[full_list['Rep'] == 'Gill']
howard_list_trx = full_list[full_list['Rep'] == 'Howard']
jardine_list_trx = full_list[full_list['Rep'] == 'Jardine']
jones_list_trx = full_list[full_list['Rep'] == 'Jones']
kivell_list_trx = full_list[full_list['Rep'] == 'Kivell']
morgan_list_trx = full_list[full_list['Rep'] == 'Morgan']
parent_list_trx = full_list[full_list['Rep'] == 'Parent']
smith_list_trx = full_list[full_list['Rep'] == 'Smith']
sorvino_list_trx = full_list[full_list['Rep'] == 'Sorvino']
thompson_list_trx = full_list[full_list['Rep'] == 'Thompson']

trx_list = [
    andrew_list_trx, 
    gil_list_trx, 
    howard_list_trx,
    jardine_list_trx,
    jones_list_trx,
    kivell_list_trx,
    morgan_list_trx,
    parent_list_trx,
    smith_list_trx,
    sorvino_list_trx,
    thompson_list_trx
]

[*] Intentando leer el archivo: /Users/kevinaleman/Desktop/Tecnic_test_KEA/jupyter/orders.csv
[ÉXITO] Archivo 'orders.csv' cargado correctamente.
[*] Filas leídas: 43. Columnas: 7.


In [ ]:

response = []
for usr, trx_list in zip(user_list, trx_list):
    sale_usr = SalesUsuarios(usr, SaleSummary(trx_list))
    response.append(asdict(sale_usr)) 

with open("integration_output.json", "w", encoding="utf-8") as f:
    json.dump(response, f, indent=4, ensure_ascii=False)

[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=-37.3159&lon=81.1496&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=-43.9509&lon=-34.4618&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=-68.6102&lon=-47.0653&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=29.4572&lon=-164.2990&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=-31.8129&lon=62.5342&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=-71.4197&lon=71.7478&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] Haciendo solicitud GET a: https://api.openweathermap.org/data/3.0/onecall?lat=24.8918&lon=21.8984&appid=6d3845d7ba9032d63278d8eb8be33f07
[*] 

FileNotFoundError: [Errno 2] No such file or directory: 'jupyter/integration_output.json'